In [ ]:
import os

In [4]:
%pwd

'd:\\Projects\\Text_Summarizer\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\Projects\\Text_Summarizer'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [8]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories


In [13]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
            config = self.config.data_transformation

            create_directories([config.root_dir])

            data_transformation_config = DataTransformationConfig(
                root_dir=config.root_dir,
                data_path=config.data_path,
                tokenizer_name=config.tokenizer_name
            )

            return data_transformation_config
        

In [1]:
pip install dataclasses

Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk


In [18]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, examples_batch):
        input_encodings = self.tokenizer(examples_batch['dialogue'], max_length=128, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(examples_batch['summary'], max_length=64, truncation=True)

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        data_samsum = load_from_disk(self.config.data_path)
        data_samsum_pt = data_samsum.map(self.convert_examples_to_features, batched=True)
        data_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, 'data_samsum_pt'))



In [19]:
try:
    config = ConfigurationManager()
    data_transformation_config = ConfigurationManager().get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e

[2025-03-29 23:05:40,092:  INFO: yaml file loaded successfully: config\config.yaml]
[2025-03-29 23:05:40,095:  INFO: yaml file loaded successfully: params.yaml]
[2025-03-29 23:05:40,095:  INFO: Directory created at artifacts]
[2025-03-29 23:05:40,106:  INFO: yaml file loaded successfully: config\config.yaml]
[2025-03-29 23:05:40,110:  INFO: yaml file loaded successfully: params.yaml]
[2025-03-29 23:05:40,112:  INFO: Directory created at artifacts]
[2025-03-29 23:05:40,114:  INFO: Directory created at artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 51915.52 examples/s]
